In [3]:
#   ___                  _                
#  / _/______ ____  ____(_)__ _______     
# / _/ __/ _ `/ _ \/ __/ (_-</ __/ _ \    
#/_//_/  \_,_/_//_/\__/_/___/\__/\___/    
# ___ _____(_)__ ___ ____  / /_(_)       
# / _ `/ __/ (_-</ _ `/ _ \/ __/ /        
# \_, /_/ /_/___/\_,_/_//_/\__/_/         
#/___/
#
#Samee Lab @ Baylor College Of Medicine
#francisco.grisanticanozo@bcm.edu

In [4]:
import psutil
import platform
from datetime import datetime

print("="*20, "System Information", "="*20)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")

==================== System Information ====================
System: Linux
Node Name: martinlab-bobbyd
Release: 4.15.0-142-generic
Version: #146-Ubuntu SMP Tue Apr 13 01:11:19 UTC 2021
Machine: x86_64


In [19]:
#Load libraries
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import sys
import pandas as pd
import warnings
import seaborn as sb
import sklearn
import tqdm
import scanpy as sc
import anndata
import sklearn
import tangram as tg

In [10]:
import torch
torch.cuda.empty_cache()
device = torch.device("cpu")

# Run Tangram

In [40]:
ad_sc = sc.read_h5ad('../Data/Tabula_Sapiens/adata_train.h5ad')

In [41]:
ad_sp = sc.read_h5ad('../Data/Tabula_Sapiens/adata_predict.h5ad')
del ad_sp.uns, ad_sp.obs, ad_sp.var, ad_sp.obsp

In [42]:
ad_sp

AnnData object with n_obs × n_vars = 10418 × 15000
    obsm: 'X_umap'

In [30]:
ad_sc, ad_sp = tg.pp_adatas(ad_sc, ad_sp, genes=None)

INFO:root:7753 marker genes shared by AnnDatas.


In [31]:
ad_map = tg.map_cells_to_space(ad_sc, ad_sp,device=device,num_epochs=500)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 7753 genes in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.019
Score: 0.802
Score: 0.810
Score: 0.812
Score: 0.813


INFO:root:Saving results..


In [32]:
ad_map.write_h5ad('../outputs/tangram_ad_map.h5ad')

# Get Tangram results

In [104]:
ad_sc = sc.read_h5ad('../Data/Tabula_Sapiens/adata_train.h5ad')

In [105]:
ad_sp = sc.read_h5ad('../Data/Tabula_Sapiens/adata_predict.h5ad')

In [106]:
ad_map = sc.read('../outputs/tangram_ad_map.h5ad')

## By mean cell probability

In [107]:
result = []

for row in range(10418):
    
    #get mapping for cell
    temp = pd.DataFrame({'celltypes':ad_sc.obs['celltype'].values,
                     'probabilities':ad_map.X[:,row]})
    
    #mean probability across all celltypes
    temp = temp.groupby('celltypes').mean()
    
    #extract max probability
    
    temp = temp.idxmax().values[0]
    
    result.append(temp)

In [108]:
ad_sp.obs['Tangram_prediction_avg'] = result

## By max cell probability

In [109]:
result = []

for row in range(10418):
    
    #get mapping for cell
    temp = pd.DataFrame({'celltypes':ad_sc.obs['celltype'].values,
                     'probabilities':ad_map.X[:,row]})
    
    #extract max probability
    max_prob = temp.iloc[temp['probabilities'].idxmax(axis=1)][0]
    
    result.append(max_prob)

In [110]:
ad_sp.obs['Tangram_prediction_max'] = result

## By sum cell probability

In [111]:
result = []

for row in range(10418):
    
    #get mapping for cell
    temp = pd.DataFrame({'celltypes':ad_sc.obs['celltype'].values,
                     'probabilities':ad_map.X[:,row]})
    
    #mean probability across all celltypes
    temp = temp.groupby('celltypes').sum()
    
    #extract max sum probability
    
    temp = temp.idxmax().values[0]
    
    result.append(temp)

In [112]:
ad_sp.obs['Tangram_prediction_maxsum'] = result

# Save predictions to file

In [113]:
ad_sp.obs[['celltype','Tangram_prediction_avg','Tangram_prediction_max','Tangram_prediction_maxsum']].to_csv('../outputs/Tabula_Sapiens_predictions_Tangram.csv')